In [15]:
## Job1: simple wordcount on term-document tuples to compute tf

### Map
# Input: (docname, contents):
# Output: ((docname,term), 1)

### Reduce
# Input: ((docname,term), 1)
# Output: ((term,docname), (N,n))

###########################################  mapper1.py

import os
import sys

### D = ?????

for line in sys.stdin:
    
    line = line.strip()
    terms = line.split(" ")
       
    path = os.environ['mapreduce_map_input_file'].split('/')
    docname = path[-1]
    
    for term in terms:
        term = term.strip('''!()-[]{};:'"\,<>./?@#$%^&*_~''').lower()
        print('%s\t%s' % (term + '_' + docname, 1))
        
        
###########################################  reducer1.py

import sys

current_file = None
current_term = None
term_list = []
current_term_count = 0
current_doc_count = 0
pair = None


for line in sys.stdin:
    
    line = line.strip()
    pair, count = line.split('\t', 1)
    
    file, term = pair.split('_', 1)
 
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue
 
    if current_file == file:
        current_doc_count += count
        if current_term == term:
            current_term_count += count
        else:
            term_list.append((term, current_term_count))
            current_term = term
            current_term_count = count
    else:
        if current_file:
            # write result to STDOUT
            for term,current_term_count in term_list:
                print ('%s\t%s' % (term+'_'+file, str(current_doc_count)+'_'+(str(current_term_count))))
        current_doc_count = count
        current_file = file
        current_term = term
        term_list = []   

if (current_file == file and current_term == term):
    for term,current_term_count in term_list: 
        print ('%s\t%s' % (term+'_'+file, str(current_doc_count)+'_'+(str(current_term_count))))
    

In [29]:
## Job2: append document frequency d to term_doc pairs

### Map
# Input:  ((term,docname), (N,n))
# Output: (term, (docname,N,n,1))

### Reduce
# Input: (term, (docname,N,n,1))
# Output: ((term,docname), (N,n,d))



###########################################  mapper2.py

import sys

for line in sys.stdin:
    
    pair,vals = line.split('\t')
    term,docname = pair.split('_',1)
    N,n = vals.split('_')
    
    try:
        n = int(n)
        N = int(N)
    except ValueError:
        continue
    
    print('%s\t%s' % (term, docname + '_'+str(N) + '_' + str(n) + '_' + str(1)))
        
        
###########################################  reducer2.py

import sys

current_term = None
doc_list = []
current_count = 0
term = None
doc = None

for line in sys.stdin:

    term,rest = line.split('\t',1)
    doc,N,n,count = rest.rsplit('_',3)
 
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue
 
    if current_term == term:
        doc_list.append((doc,N,n))
        current_count += count
    else:
        if current_term:
            for document,N,n in doc_list:
                print ('%s\t%s' % (current_term + '_' + document, str(N)+'_' + str(n) + '_' + str(current_count))
            doc_list = []   
            n_list = []
            
        current_count = count
        current_term = term
        doc_list.append((doc,N,n))
        
        

if current_term == term:
    for document,N,n in doc_list:
        print ('%s\t%s' % (current_term + '_' + document, str(N)+'_' + str(n) + '_' + str(current_count))

In [30]:
## Job3: compute tf-idfs

### Map
# Input:  ((term,docname), (N,n,d))
# Output: ((term,docname), tfidf)

### Reduce ----pass
# Input:((term,docname), tfidf)
# Output: ((term,docname), tfidf)

###########################################  mapper3.py

import sys
import math

for line in sys.stdin:
    
    pair,vals = line.split('\t',1)
    N,n,d = rest.split('_',2)
    
    try:
        N = int(N)
        n = int(n)
        d = int (d)
    except ValueError:
        continue
        
        
    ## to calculate these with normalizing terms we need to compute:
 
    
    tf = n/N  
    idf = math.log(10000/(1+d))
    tfidf = tf * idf
    
    print('%s\t%s' % (pair, tfidf))
    
    
###########################################  reducer3.py

import sys


for line in sys.stdin:
    print (line)

    

In [ ]:
minutes Jan 31st meeting:
    - computation of Nj and D
    - 